In [97]:
import pandas as pd
from pathlib import Path
import numpy as np
from sklearn.model_selection import cross_validate, train_test_split, RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler 
from scipy.stats import randint, loguniform

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression



In [98]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
np.set_printoptions(suppress=True, precision=4)

In [99]:
df = pd.read_csv("starting_data.csv")
print(df.shape)
df.head()

(554, 18)


,team,season,corsiPercentage,fenwickPercentage,xGoalsPercentage,xOnGoalFor_PerGame,xPlayContinuedInZoneFor_PerGame,xPlayContinuedOutsideZoneFor_PerGame,xPlayContinuedInZoneAgainst_PerGame,xPlayContinuedOutsideZoneAgainst_PerGame,flurryScoreVenueAdjustedxGoalsFor_PerGame,lowDangerShotsFor_PerGame,lowDangerxGoalsFor_PerGame,lowDangerGoalsFor_PerGame,lowDangerShotsAgainst_PerGame,lowDangerxGoalsAgainst_PerGame,lowDangerGoalsAgainst_PerGame,playoff
0,MIN,2008,0.48,0.48,0.48,26.951341,15.443049,11.653902,16.666707,12.588902,2.230976,30.353659,0.887561,0.902439,33.219512,0.992927,0.853659,0
1,BOS,2008,0.50,0.50,0.51,29.458889,16.346296,12.585556,15.995185,12.556420,2.693704,30.901235,0.864815,1.086420,30.913580,0.878395,0.691358,1
2,ARI,2008,0.46,0.47,0.47,27.009383,15.300123,11.586049,17.462346,13.736173,2.328889,30.222222,0.872346,0.790123,35.012346,1.011111,0.876543,0
3,L.A,2008,0.51,0.51,0.51,29.853659,17.133659,12.453171,16.105000,11.953659,2.633780,32.365854,0.964146,0.719512,31.109756,0.936951,0.865854,0
4,COL,2008,0.47,0.48,0.49,26.916951,14.914756,11.198171,15.753659,12.303780,2.378659,29.646341,0.936951,0.780488,32.390244,0.998293,1.073171,0


In [100]:
df_train = df[df["season"] != 2025]
df_test = df[df["season"] == 2025]

In [101]:
df_test.head()

,team,season,corsiPercentage,fenwickPercentage,xGoalsPercentage,xOnGoalFor_PerGame,xPlayContinuedInZoneFor_PerGame,xPlayContinuedOutsideZoneFor_PerGame,xPlayContinuedInZoneAgainst_PerGame,xPlayContinuedOutsideZoneAgainst_PerGame,flurryScoreVenueAdjustedxGoalsFor_PerGame,lowDangerShotsFor_PerGame,lowDangerxGoalsFor_PerGame,lowDangerGoalsFor_PerGame,lowDangerShotsAgainst_PerGame,lowDangerxGoalsAgainst_PerGame,lowDangerGoalsAgainst_PerGame,playoff
522,NYI,2025,0.49,0.50,0.48,31.238571,16.920476,12.004762,16.820952,11.972381,2.890714,31.309524,0.941905,1.119048,30.380952,0.917857,0.928571,1
523,NYR,2025,0.49,0.49,0.46,30.110455,16.524545,11.647273,17.248409,12.328636,2.713409,30.340909,0.844545,0.840909,32.704545,0.998864,0.886364,0
524,PIT,2025,0.50,0.50,0.51,31.960244,17.122195,12.483415,17.075366,12.207561,3.078293,32.097561,0.993659,1.317073,31.439024,0.945610,1.341463,1
525,LAK,2025,0.52,0.52,0.51,31.795000,17.153250,12.403000,15.941750,11.814500,2.822750,32.700000,0.919500,0.875000,29.375000,0.855500,0.900000,1
526,UTA,2025,0.52,0.53,0.51,31.569767,17.066977,12.144651,14.978140,10.759535,2.901628,31.674419,0.952791,1.069767,27.255814,0.827907,0.953488,0


In [102]:
team_key = df_test.loc[:, "team"]
df_train = df_train.drop(columns=["team"])
df_test = df_test.drop(columns=["team"])

### i think i forgot to drop season

In [103]:
team_key

522    NYI
523    NYR
524    PIT
525    LAK
526    UTA
527    TOR
528    MIN
529    VAN
530    PHI
531    CAR
532    WSH
533    NJD
534    CGY
535    SEA
536    WPG
537    STL
538    OTT
539    ANA
540    DET
541    SJS
542    DAL
543    BUF
544    VGK
545    EDM
546    BOS
547    CHI
548    FLA
549    CBJ
550    COL
551    MTL
552    NSH
553    TBL
Name: team, dtype: object

In [104]:
X_train = df_train.drop(columns=["playoff"])
y_train = df_train.loc[:, "playoff"]
X_test = df_test.drop(columns=["playoff"])
y_test = df_test.loc[:, "playoff"]


In [105]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(522, 16) (522,) (32, 16) (32,)


In [106]:
pipe = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000))
result = pd.DataFrame(cross_validate(pipe, X_train, y_train, cv=10, return_train_score=True))
result.sort_values("test_score", ascending=False)

,fit_time,score_time,test_score,train_score
9,0.009320,0.001883,0.903846,0.851064
2,0.014359,0.001957,0.903846,0.846809
8,0.008809,0.001189,0.865385,0.848936
4,0.032444,0.003289,0.846154,0.855319
1,0.010621,0.001853,0.830189,0.861407
0,0.011207,0.001972,0.811321,0.857143
5,0.021950,0.002715,0.807692,0.865957
3,0.010385,0.001891,0.807692,0.859574
7,0.011536,0.001230,0.807692,0.861702
6,0.007526,0.002034,0.807692,0.861702


In [107]:
result["test_score"].mean().item()

0.8391509433962264

In [119]:
param_choices = {
    "logisticregression__C": loguniform(1e-3, 1e3)
}
random_search = RandomizedSearchCV(pipe, param_choices, n_iter=100, verbose=1, n_jobs=-1)
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


,"estimator estimator: estimator objectAn object of that type is instantiated for each grid point.This is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Pipeline(step..._iter=1000))])
,"param_distributions param_distributions: dict or list of dictsDictionary with parameters names (`str`) as keys and distributionsor lists of parameters to try. Distributions must provide a ``rvs``method for sampling (such as those from scipy.stats.distributions).If a list is given, it is sampled uniformly.If a list of dicts is given, first a dict is sampled uniformly, andthen a parameter is sampled using that dict as above.",{'logisticregression__C': <scipy.stats....002AB203C5F50>}
,"n_iter n_iter: int, default=10Number of parameter settings that are sampled. n_iter tradesoff runtime vs quality of the solution.",100
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.If None, the estimator's score method is used.",None
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given the ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``RandomizedSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-f

In [120]:
pd.DataFrame(random_search.cv_results_)[["mean_test_score", "param_logisticregression__C", "mean_fit_time", "rank_test_score"]].set_index("rank_test_score").sort_index()

,mean_test_score,param_logisticregression__C,mean_fit_time
rank_test_score,,,
1,0.848590,0.189278,0.005731
2,0.848553,0.245451,0.005176
3,0.846667,0.419197,0.003228
3,0.846667,0.386152,0.003815
5,0.846667,0.154756,0.003462
...,...,...,...
94,0.750897,0.002495,0.004780
94,0.750897,0.002389,0.002776
98,0.748974,0.002247,0.003256


In [121]:
final_pipe = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000, C=0.189278))
final_pipe.fit(X_train, y_train)
final_pipe.score(X_test, y_test)

0.6875

In [134]:
pd.DataFrame(y_test.values, index=team_key) == pd.DataFrame(final_pipe.predict(X_test), index=team_key)

,0
team,
NYI,False
NYR,True
PIT,True
LAK,False
UTA,False
TOR,True
MIN,True
VAN,True
PHI,False


In [138]:
pd.DataFrame(final_pipe.predict_proba(X_test), index=team_key).sort_values(1, ascending=False)

,0,1
team,,
COL,0.000079,0.999921
TBL,0.013682,0.986318
DAL,0.024561,0.975439
WSH,0.056231,0.943769
CAR,0.067202,0.932798
MIN,0.092319,0.907681
OTT,0.092398,0.907602
BUF,0.287679,0.712321
UTA,0.321767,0.678233
